# Notebook for tf-idf information retrieval

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from src import init_data
from arqmath_code.topic_file_reader import Topic
from arqmath_code.Entities.Post import Post
import re

In [2]:
topic_reader, data_reader = init_data(task=1)

reading users
reading comments
reading votes
reading post links
reading posts


In [3]:
test_topic: Topic = topic_reader.get_topic('A.301')

## Remove html tags
As we are not interested in their word counts in the actual tf-idf vectors

In [4]:
test_topic.title = re.sub(r"</?(p|span)[^>]*>", "", test_topic.title)
test_topic.question = re.sub(r"</?(p|span)[^>]*>", "", test_topic.question)
test_topic.question

"Suppose $A$ is a $m\\times n$ matrix. Then Prove that, $\\begin{equation*} \\|A\\|_2\\leq \\sqrt{\\|A\\|_1 \\|A\\|_{\\infty}} \\end{equation*}$ I have proved the following relations: $\\begin{align*} \\frac{1}{\\sqrt{n}}\\|A\\|_{\\infty}\\leq \\|A\\|_2\\leq\\sqrt{m}\\|A\\|_{\\infty}\\\\ \\frac{1}{\\sqrt{m}}\\|A\\|_{1}\\leq \\|A\\|_2\\leq\\sqrt{n}\\|A\\|_{1} \\end{align*}$ Also I feel that somehow Holder's inequality for the special case when $p=1$ and $q=\\infty$ might be useful.But I couldn't prove that. Edit: I would like to have a prove that do not use the information that $\\|A\\|_2=\\sqrt{\\rho(A^TA)}$ Usage of inequalities like Cauchy Schwartz or Holder is fine. "

In [8]:
def remove_html_tags_from_post(post: Post) -> Post:
    post.body = re.sub(r"</?(p|span)[^>]*>", "", post.body)
    return post

## Retrieve Subset of Document Collection by tags
Equivalent to a binary retrieval of documents by only considering tags

In [7]:
potential_questions = [question for tag in test_topic.lst_tags for question in data_reader.get_question_of_tag(tag=tag)]
len(potential_questions)

134703

In [10]:
for question in potential_questions:
    question.title = re.sub(r"</?(p|span)[^>]*>", "", question.title)
    question = remove_html_tags_from_post(post=question)

potential_questions[0].body

"I was reading up on the Fibonacci Sequence,  \\text {{1,1,2,3,5,8,13,....}} when I've noticed some were able to calculate specific numbers. So far I've only figured out creating an array and counting to the value, which is incredibly simple, but I reckon I can't find any formula for calculating a Fibonacci number based on it's position.  Is there a way to do this? If so, how are we able to apply these formulas to arrays? "

## Create TF-IDF Vectors for Question Titles

In [15]:
# Basic example from sklearn
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(corpus)
print(x)
vectorizer.get_feature_names_out()

  (0, 1)	0.46979138557992045
  (0, 2)	0.5802858236844359
  (0, 6)	0.38408524091481483
  (0, 3)	0.38408524091481483
  (0, 8)	0.38408524091481483
  (1, 5)	0.5386476208856763
  (1, 1)	0.6876235979836938
  (1, 6)	0.281088674033753
  (1, 3)	0.281088674033753
  (1, 8)	0.281088674033753
  (2, 4)	0.511848512707169
  (2, 7)	0.511848512707169
  (2, 0)	0.511848512707169
  (2, 6)	0.267103787642168
  (2, 3)	0.267103787642168
  (2, 8)	0.267103787642168
  (3, 1)	0.46979138557992045
  (3, 2)	0.5802858236844359
  (3, 6)	0.38408524091481483
  (3, 3)	0.38408524091481483
  (3, 8)	0.38408524091481483


array(['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third',
       'this'], dtype=object)

In [14]:
potential_question_titles = [question.title for question in potential_questions]
potential_question_titles

['How are we able to calculate specific numbers in the Fibonacci Sequence?',
 'Why is the matrix-defined Cross Product of two 3D vectors always orthogonal?',
 'What Is An Inner Product Space?',
 'Calculating an Angle from 2 points in space',
 'Real world uses of Quaternions?',
 'Understanding Dot and Cross Product',
 'Recasting points from one vector space to another',
 'Why are 3D transformation matrices 4 \\times 4 instead of 3 \\times 3?',
 'Intuitive reasoning behind the Chain Rule in multiple variables?',
 'Can non-linear transformations be represented as Transformation Matrices?',
 'How can I tell which matrix decomposition to use for OLS?',
 'What is the most efficient way to determine if a matrix is invertible?',
 'Balance chemical equations without trial and error?',
 "What's an intuitive way to think about the determinant?",
 'Are there variations on least-squares approximations?',
 'How to visualize a rank-2 tensor?',
 'What is linear programming?',
 'Determinants and volume

In [24]:
training_set = potential_question_titles.copy()
training_set.append(test_topic.title)

vectorizer = TfidfVectorizer()
vectorizer.fit(training_set)
question_vector = vectorizer.transform([test_topic.title])
print(type(question_vector))
print(question_vector)
print(len(vectorizer.get_feature_names_out()))

<class 'scipy.sparse._csr.csr_matrix'>
  (0, 10166)	0.11448280664760913
  (0, 9964)	0.8020032049381629
  (0, 9104)	0.2097147300338194
  (0, 7561)	0.367589433206298
  (0, 7487)	0.22594235867134732
  (0, 2426)	0.28995630500843966
  (0, 1717)	0.1716323937862452
16014
